# Analyse de cohorte

In [4]:
import pandas as pd
import datetime

In [2]:
df = pd.read_csv("/Users/stratospheric/Documents/Retail Analysis/Online Retail RFM Full.csv")

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Label
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,Fidele Fuyant
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Fidele Fuyant
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,Fidele Fuyant
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Fidele Fuyant
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,Fidele Fuyant


In [10]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'],errors='coerce')


In [83]:
months = []
years = []


for date in df['InvoiceDate'] :
    month = date.month
    year = date.year
    months.append(month)
    years.append(year)

df['Year'] = years
df['Month'] = months

In [84]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']


In [97]:
df['Date'] = df['Year'].astype(str) + "-" + df['Month'].astype(str)


In [98]:
df['Date'] = pd.to_datetime(df['Date'])

In [111]:
first_order = df.groupby('CustomerID')['Date'].min().reset_index().rename(columns={'Date':'Cohort'})

first_order.head()

,CustomerID,Cohort
0,12347.0,2010-12-01
1,12348.0,2010-12-01
2,12349.0,2011-11-01
3,12350.0,2011-02-01
4,12352.0,2011-02-01


In [147]:
cohort_df = df.groupby(['Date','CustomerID']).agg({
    'InvoiceNo':'nunique',
    'TotalPrice':'sum'})

cohort_df = cohort_df.reset_index()
cohort_df.head()

,Date,CustomerID,InvoiceNo,TotalPrice
0,2010-12-01,12347.0,1,650.89
1,2010-12-01,12348.0,1,52.80
2,2010-12-01,12370.0,2,1303.82
3,2010-12-01,12377.0,1,1001.52
4,2010-12-01,12383.0,1,520.32


In [148]:
cohort_df = pd.merge(cohort_df,first_order, on='CustomerID')

In [149]:
cohort_df.head()

,Date,CustomerID,InvoiceNo,TotalPrice,Cohort
0,2010-12-01,12347.0,1,650.89,2010-12-01
1,2011-01-01,12347.0,1,437.14,2010-12-01
2,2011-04-01,12347.0,1,325.00,2010-12-01
3,2011-06-01,12347.0,1,351.82,2010-12-01
4,2011-08-01,12347.0,1,342.66,2010-12-01


In [160]:
cohort_grouped = cohort_df.groupby(['Cohort','Date']).agg({
    'InvoiceNo':'sum',
    'TotalPrice':'sum',
    'CustomerID':'nunique'})#.reset_index()

new_names = {'CustomerID':'No_Customers','InvoiceNo':'No_Orders','TotalPrice':'NetSales'}
cohort_grouped = cohort_grouped.rename(columns=new_names)
cohort_grouped = cohort_grouped.sort_values(by=['Cohort','Date'],ascending=True)

cohort_grouped.head()

No_Orders   NetSales  No_Customers
Cohort     Date                                          
2010-12-01 2010-12-01       1270  286014.61           833
           2011-01-01        460  117439.05           302
           2011-02-01        395   91136.04           261
           2011-03-01        500  113963.31           307
           2011-04-01        438   99942.26           295

In [166]:
#Marge de la boutique
margin = 0.41

#Calcul du gross profit en prenant en compte la marge
cohort_grouped['GrossProfit'] = cohort_grouped['NetSales']*margin

cohort_df.groupby(['Cohort','Date'])

#Calcul des profits cumulés
cohort_grouped['CumulatedGrossProfit'] = cohort_grouped.groupby('Cohort')['GrossProfit'].cumsum()
cohort_grouped.head()



No_Orders   NetSales  No_Customers  GrossProfit  \
Cohort     Date                                                          
2010-12-01 2010-12-01       1270  286014.61           833  117265.9901   
           2011-01-01        460  117439.05           302   48150.0105   
           2011-02-01        395   91136.04           261   37365.7764   
           2011-03-01        500  113963.31           307   46724.9571   
           2011-04-01        438   99942.26           295   40976.3266   

                       CumulatedGrossProfit  
Cohort     Date                              
2010-12-01 2010-12-01           117265.9901  
           2011-01-01           165416.0006  
           2011-02-01           202781.7770  
           2011-03-01           249506.7341  
           2011-04-01           290483.0607

Pour calculer la LTV il suffit de diviser les profits cumulés par le nombre de client par cohortes.

In [176]:
cohort_grouped['LTV'] = round(cohort_grouped['CumulatedGrossProfit']/(cohort_grouped.groupby('Cohort')['No_Customers'].first()),2)

In [184]:
cohort_grouped = cohort_grouped.reset_index('Date')

In [231]:
cohort_grouped['Month'] = cohort_grouped.groupby('Cohort')['Date'].rank(method='first')
cohort_grouped['Retention'] = cohort_grouped['No_Customers']/(cohort_grouped.groupby('Cohort')['No_Customers'].first())

#Limite la LTV à 12 mois
ltv12 = cohort_grouped[cohort_grouped['Month'] <=13]

In [225]:
#Export des resultats
cohort_grouped.to_csv("Cohort Analysis.csv")

Nous allons maintenant créer une pivot table pour visualiser la LTV par cohorte sur 12 mois.

In [232]:
#Pivot Table pour simplifier les résultats
ltv12 = ltv12.pivot_table(index='Cohort',columns='Month', values='LTV')

In [233]:
ltv12

Month,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0
Cohort,,,,,,,,,,,,,
2010-12-01,140.78,198.58,243.44,299.53,348.72,411.66,467.67,524.70,581.33,661.59,735.00,848.01,887.3
2011-01-01,119.68,146.30,175.74,199.83,243.18,282.47,317.86,352.83,400.54,460.79,529.09,542.97,NaN
2011-02-01,117.69,133.63,157.09,192.68,225.49,250.87,284.91,332.02,372.79,418.17,425.09,NaN,NaN
2011-03-01,120.53,137.12,168.67,193.52,220.98,246.34,285.52,324.66,366.55,374.58,NaN,NaN,NaN
2011-04-01,115.58,141.15,161.22,184.74,206.58,238.58,269.32,303.81,311.09,NaN,NaN,NaN,NaN
2011-05-01,107.77,123.87,141.43,159.85,191.75,228.22,264.37,276.26,NaN,NaN,NaN,NaN,NaN
2011-06-01,103.99,117.83,132.93,164.03,196.60,241.22,251.28,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,109.45,125.39,156.77,190.97,225.59,238.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,138.33,170.98,231.53,298.47,324.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Voila une bonne base pour créer un dashboard via Looker ou Tableau.